In [ ]:
!ls
!ls data

In [10]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input

from keras.applications import MobileNetV2
from keras.applications import VGG16




In [2]:
print(tf.__version__)
print(keras.__version__)

2.0.0
2.2.4-tf


In [3]:
from keras.models import Model

model1 = MobileNetV2(weights="imagenet")
model2 = Model(inputs=model1.inputs, outputs=model1.layers[-3].output)

In [4]:
model1.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [16]:
(model2.layers[-1].output_shape)[1:]

(7, 7, 1280)

In [ ]:
#model_vgg = VGG16(include_top=True)
#model_vgg.summary()

In [6]:
import pandas as pd

sample = pd.read_table("data/sample.txt",header=None)
train_triplets = pd.read_table("data/train_triplets.txt",delimiter=" ",header=None,dtype=str)
test_triplets = pd.read_table("data/test_triplets.txt",delimiter=" ",header=None,dtype=str)

In [ ]:
train_triplets


In [ ]:
test_triplets

In [ ]:
test_triplets[0][1]

In [22]:
a = [3]+[4]

In [23]:
a

[3, 4]

In [32]:
def image_pred(image_code,model):
    image_path = 'data/food/'+image_code+'.jpg'
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    print("yhat:")
    print(np.shape(yhat))
    #yhat = yhat[0]

    return(yhat)


def triplet_pred(triplet,model):
    triplet_feats_shape = tuple([3]+list((model.layers[-1].output_shape)[1:]))
    print(triplet_feats_shape)
    triplet_feats = np.zeros(triplet_feats_shape)
    for i in range(len(triplet)):
        triplet_feats[i] = image_pred(triplet[i],model)

    dot_01 = np.dot(triplet_feats[0].flatten(),triplet_feats[1].flatten())
    dot_02 = np.dot(triplet_feats[0].flatten(),triplet_feats[2].flatten())
    if dot_01 > dot_02:
        return(1)
    else:
        return(0)


In [8]:
train_triplets.shape

(59515, 3)

In [33]:
n = train_triplets.shape[0]
y = np.zeros(n)

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

for i in range(100):
    triplet = train_triplets.loc[i].values
    y[i] = triplet_pred(triplet,model2)
    print(i)
    print("\n")
    
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 13:40:01
(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
0


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
1


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
2


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
3


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
4


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
5


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
6


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
7


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
8


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
9


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
10


(3, 7, 7, 1280)
yhat:
(1, 7, 

yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
94


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
95


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
96


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
97


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
98


(3, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
yhat:
(1, 7, 7, 1280)
99


Current Time = 13:40:49


In [34]:
np.sum(y[:100])

48.0

In [40]:
now = datetime.now()
current_time = now.strftime("%m_%d_%H_%M_%S")
print("Current Time =", current_time)

Current Time = 05_20_14_23_30


In [43]:
import os


os.mkdir("jobs")
os.mkdir("jobs/job_"+current_time)

In [48]:
current_job_path = os.path.join("jobs","job_"+current_time)

In [49]:
current_job_path 

'jobs/job_05_20_14_23_30'

In [45]:
os.path.exists("job")

False

In [ ]:

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input
#from keras.applications.vgg16 import decode_predictions
#from keras.applications.vgg16 import VGG16
# load an image from file
image = load_img('data/food/00000.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)

In [ ]:
yhat = model.predict(image)

In [ ]:
np.shape(yhat.flatten())